In [1]:
# Google Colab Setup
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

from keras.layers import Input, LeakyReLU
from keras.models import Model, Sequential
from keras.layers.core import Reshape, Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, UpSampling2D
from keras.datasets import mnist
from keras.optimizers import Adam
from keras import backend as K
from keras import initializers

In [0]:
os.chdir('/content/drive/My Drive/app/DCGAN-MNIST/')
os.environ["KERAS_BACKEND"] = 'tensorflow'
K.set_image_dim_ordering('th')

np.random.seed(1000)
randomDim = 100

In [6]:
#Load MNIST Data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = X_train[:, np.newaxis, :, :]

11493376/11490434 [==============================] - 0s 0us/step


In [7]:
# Optimizer
adam = Adam(lr = 0.0002, beta_1 = 0.5)

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
# Generator Network
generator = Sequential()
generator.add(Dense(128 * 7 * 7, input_dim = randomDim, kernel_initializer = initializers.RandomNormal(stddev = 0.02)))
generator.add(LeakyReLU(0.2))
generator.add(Reshape((128, 7, 7)))
generator.add(UpSampling2D(size = (2, 2)))
generator.add(Conv2D(64, kernel_size = (5, 5), padding = 'same'))
generator.add(LeakyReLU(0.2))
generator.add(UpSampling2D(size = (2, 2)))
generator.add(Conv2D(1, kernel_size = (5, 5), padding = 'same', activation = 'tanh'))
generator.compile(loss = 'binary_crossentropy', optimizer = 'adam')

In [0]:
# Discriminator Network
